In [82]:
# Import packages
from logging.config import valid_ident
from pydoc import resolve
import ee
import pandas as pd
import numpy as np
import time
from geetools import batch
from tqdm import tqdm
from argparse import ArgumentParser
import requests
import math
import logging
import multiprocessing
import requests
from retry import retry
import multiprocessing
import os
import csv
from functools import partial
import folium

In [83]:
# Personal credentials
service_account = "sentinel2@uhi-causal-model.iam.gserviceaccount.com"
json_key = "uhi-causal-model-a7e891b460a5.json"

In [84]:
# Initialize the library
ee.Initialize(ee.ServiceAccountCredentials(service_account, json_key), opt_url='https://earthengine-highvolume.googleapis.com')

In [85]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [86]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data © Google Earth Engine',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [87]:
# Look for bounding box at a specific location
def bounding_box(lat, lon, size, res):

  earthRadius = 6371000
  angularDist = math.degrees(0.5 * ((size * res) / earthRadius))
  osLat = angularDist
  osLon = angularDist
  xMin = lon - osLon
  xMax = lon + osLon
  yMin = lat - osLat
  yMax = lat + osLat
  return xMin, xMax, yMin, yMax

In [88]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [89]:
# Import dataset
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")

In [90]:
# Define date
startDate = '2019-07-29'
endDate = '2019-07-30'

In [113]:
lat,lon = 42.325347,-71.120802
height = 512
res = 60
RGB_VIZ = {'min':0, 'max':255, 'bands':['TCI_R', 'TCI_G', 'TCI_B']}

xMin, xMax, yMin, yMax = bounding_box(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])

filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['TCI_R', 'TCI_G', 'TCI_B'])
image = filtered.median().clip(geometry)
map = folium.Map(location=[lat, lon], zoom_start=11)
map.add_ee_layer(image, RGB_VIZ, 'rgb')
display(map)

In [103]:
lat=42.325347
lon=-71.120802 
dataset=sentinel 
startDate='2019-07-29'
endDate='2019-07-30'
height=512
width=512

RGB = ['B4', 'B3', 'B2']
RGB_VIZ = {'min':[0,0,0], 'max':[255,255,255], 'bands':RGB, 'opacity': 0.8}
res = 30
crs = 'EPSG:3857'
crs2 = 'EPSG:4326'
fileFormat = 'GeoTIFF'
cloud_percent = 20 
cloud_filter = 10

description = f'Boston'
xMin, xMax, yMin, yMax = bounding_box(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])


filtered = dataset.filterDate(startDate, endDate).filterBounds(geometry).select(RGB)

image = filtered.median().clip(geometry)

# imgVis = image.visualize(bands = RGB, min = 0, max = 0.5)
# OPTIONAL: just to view the map, to comment out when exporting more than 1 image
map = folium.Map(location=[lat, lon], zoom_start=12)
map.add_ee_layer(image, RGB_VIZ, 'rgb')
display(map)